In [21]:
import pandas as pd
import numpy as np
import pickle

In [22]:
# `fundamentals_dict` is a dictionary containing stock tickers
# as keys, and a dataframe of fundamental metrics (time series) 
# for each stock
with open('data/cleaned_stock_fundamentals.pkl', 'rb') as f:
    fundamentals_dict = pickle.load(f)

stock_prices_df = pd.read_csv('data/cleaned_stock_prices.csv', index_col='Date', parse_dates=True)
stock_log_rets_df = np.log(stock_prices_df).diff()

In [23]:
prices_with_fundamentals_dict = {}

for stock in fundamentals_dict.keys():
    prices_with_fundamentals_dict[stock] = fundamentals_dict[stock]
    prices_with_fundamentals_dict[stock]['stock'] = stock
    prices_with_fundamentals_dict[stock]['q_price'] = stock_prices_df[stock]
    prices_with_fundamentals_dict[stock]['q_log_return'] = stock_log_rets_df[stock]
    rearrange = ['stock', 'q_price', 'q_log_return']
    features = [f for f in prices_with_fundamentals_dict[stock].columns if f not in rearrange]
    prices_with_fundamentals_dict[stock] = prices_with_fundamentals_dict[stock][rearrange + features]


master_df = pd.concat(prices_with_fundamentals_dict.values()).dropna()
master_df

,stock,q_price,q_log_return,peRatio,marketCap,priceToSalesRatio,ptbRatio,enterpriseValueOverEBITDA,netIncomePerShare,roe,returnOnTangibleAssets,freeCashFlowPerShare,operatingCashFlowPerShare,currentRatio,debtToEquity,capexToOperatingCashFlow,grahamNumber,incomeQuality
Date,,,,,,,,,,,,,,,,,,
2000-06-30,ABT,9.700628,0.060474,11.309541,3.099728e+10,9.197588,3.787364,35.203933,0.442105,0.083721,0.045790,0.631377,0.817871,1.646991,0.205514,-0.228024,7.247701,1.849949
2000-09-30,ABT,11.676104,0.185354,12.627429,3.305452e+10,9.962497,3.975928,39.075613,0.422691,0.078716,0.052059,0.310345,0.437845,1.627245,0.195262,-0.291200,7.146319,1.035852
2000-12-31,ABT,12.687328,0.083059,11.160536,3.363335e+10,9.078586,3.924130,33.823474,0.486984,0.087902,0.054880,0.301572,0.500418,1.716387,0.181524,-0.397360,7.791241,1.027587
2001-03-31,ABT,12.786471,0.007784,-36.633587,3.276698e+10,9.204520,4.045560,17.135238,-0.144539,-0.027608,-0.013597,-0.215379,-0.062333,0.752273,0.869584,2.455285,4.126275,0.431254
2001-06-30,ABT,12.382228,-0.032125,15.779635,3.339274e+10,8.146321,3.901749,35.938615,0.341421,0.061816,0.031271,0.927301,1.027083,1.074939,0.882505,-0.097151,6.513739,3.008260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,ZION,48.784012,-0.217365,8.767373,7.610080e+09,8.727156,1.620545,18.025506,1.450263,0.046210,0.002482,0.801989,1.149517,1.507106,0.752129,-0.302326,32.001645,0.792627
2022-12-31,ZION,49.919029,0.023000,6.436628,7.312009e+09,7.400819,1.494382,25.873286,1.909385,0.058042,0.003210,3.079219,3.321254,4.884402,1.584100,-0.072874,37.593536,1.739437
2023-03-31,ZION,46.711250,-0.066417,5.429031,4.430089e+09,4.101934,0.854570,3.363459,1.378239,0.039352,0.002331,1.283654,1.493092,1.263032,2.466628,-0.140271,32.955895,1.083333


In [24]:
def drop_stocks_with_zeros(df, threshold):
    unique_stocks = df['stock'].unique()
    stocks_to_drop = []

    for stock in unique_stocks:
        stock_data = df[df['stock'] == stock]
        for column in stock_data.columns:
            if column not in ['Date', 'stock']:
                zero_count = (stock_data[column] == 0).sum()
                if zero_count / len(stock_data) > threshold:
                    stocks_to_drop.append(stock)
                    break  # No need to check further columns for this stock

    return df[~df['stock'].isin(stocks_to_drop)]

master_df = drop_stocks_with_zeros(master_df, threshold=0.01)

master_df.to_csv('data/master.csv')